In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import torchvision.models as models
from torch.utils.data.sampler import SubsetRandomSampler
from torchaudio.datasets import SPEECHCOMMANDS

import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

from dataset import SubsetSC, small_dataset
from models import *
from utils import *
import argparse

## Setting mode and device

In [3]:
hard_split = False #wether to use hard split
mode = 'human' # define task 'word' and 'human' can be possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

## Define Dataset

In [4]:
dataset = small_dataset('./data_reco', mode, hard_split)

if hard_split: # for hard split, do not change this value
    validation_split = 0.111112
else:
    validation_split = 0.2

dataset_size = len(dataset)
shuffle_dataset = True


train_indices = []
val_indices = []

# if hard split, It split data not to include same data(same word and same people)
# both in training and valid
if hard_split:
    for i, data in enumerate(dataset):
        if (data[1][1] != '3') or not(data[1][0] in ['before', 'close', 'continue', 'fast']):
            train_indices.append(i)
        else:
            val_indices.append(i)
    dataset.remove()

# If not hard split randomly split data
else:
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset:
        np.random.seed(214)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

labels = dataset.to_index()
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)



batch_size = 8
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=False,
                                           collate_fn=collate_fn, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=False,
                                           collate_fn=collate_fn, sampler=valid_sampler)

## Basic Setting such as transform and scaling

In [5]:
# Basoc Settings
sample_rate = dataset[0][2]
new_sample_rate = 1000
waveform = dataset[0][0]

# to spectrogram and scaling
transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)
spec = torchaudio.transforms.Spectrogram(n_fft=64).to(device)
data = next(iter(train_loader))
db = torchaudio.transforms.AmplitudeToDB()
m = db(spec(data[0].cuda())).mean(axis=0)
s = db(spec(data[0].cuda())).std(axis=0)
transformed = torchaudio.transforms.Spectrogram(n_fft=64)(waveform)

## model define

In [7]:
# Model definition
model = FC_Net(n_input=transformed.shape[0], n_output=len(labels), batch_size=batch_size)
model.to(device)
model.train()
n = count_parameters(model)
print("Number of parameters: %s" % n)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

log_interval = 300
n_epoch = 100
max_patient = 50

pbar_update = 1 / (len(train_loader) + len(test_loader))
losses = []
# The transform needs to live on the same device as the model and the data.
transform = transform.to(device)

Number of parameters: 166348


## define Basic Train and test

In [8]:
def train(model, epoch, log_interval):
    pbar = tqdm(train_loader)
    model.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data = data.to(device)
        target = target.to(device)

        data = spec(data)
        data = db(data)
        data = (data - m)/s
        output = model(data)
        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)
        loss = F.nll_loss(output.squeeze(), target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print training stats
        if batch_idx % log_interval == 0:
            print(f"\nTrain Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)*(1-validation_split)} ({100. * batch_idx / len(train_loader)*(1-validation_split):.0f}%)]\tLoss: {loss.item():.6f}")

        # update progress bar
        pbar.update(pbar_update)
        # record loss
        losses.append(loss.item())
    print(f"\nTrain Epoch: {epoch}\tAccuracy: {correct}/{len(train_loader.dataset)*(1-validation_split)} ({100. * correct / (len(train_loader.dataset)*(1-validation_split)):.0f}%)\n")
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()

def test(model, epoch, test_loader_):
    model.eval()
    correct = 0
    pbar = tqdm(test_loader_)
    for data, target in pbar:
        data = data.to(device)
        target = target.to(device)        
        data = spec(data)
        data = db(data)
        data = (data - m)/s
        output = model(data)
        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)

        # update progress bar
        pbar.update(pbar_update)

    print(f"\nTest Epoch: {epoch}\tAccuracy: {correct}/{len(test_loader_.dataset)*validation_split} ({100. * correct / (len(test_loader_.dataset)*validation_split):.0f}%)\n")
    accuracy = 100. * correct / (len(test_loader_.dataset)*validation_split)
    return  accuracy

In [9]:
with tqdm(total=n_epoch) as pbar:
    max_acc = 0
    early_count = 0
    max_epoch = 0
    for epoch in range(1, n_epoch + 1):
        train(model, epoch, log_interval)
        accuracy = test(model, epoch, test_loader)
        if max_acc < accuracy:
            max_acc = accuracy
            early_count = 0
            max_epoch = epoch
        else:
            early_count += 1
            if early_count == max_patient:
                break
        scheduler.step()
print(f'\n Max Test Accuracy is {max_acc} when epoch is {max_epoch}') 

  0%|          | 0/88 [00:00<?, ?it/s]
  0%|          | 0.20909090909090913/88 [00:00<00:42,  2.08it/s]
 27%|██▋       | 23.463636363636333/88 [00:00<00:00, 135.22it/s]


Train Epoch: 1 [0/702.4000000000001 (0%)]	Loss: 138.145248



100%|██████████| 88/88 [00:00<00:00, 251.16it/s]                

100%|██████████| 22/22 [00:00<00:00, 566.96it/s]

  0%|          | 0/88 [00:00<?, ?it/s]
  0%|          | 0.23636363636363641/88 [00:00<00:38,  2.30it/s]


Train Epoch: 1	Accuracy: 167/702.4000000000001 (24%)


Test Epoch: 1	Accuracy: 64/175.60000000000002 (36%)


Train Epoch: 2 [0/702.4000000000001 (0%)]	Loss: 52.314659



 30%|███       | 26.472727272727244/88 [00:00<00:00, 151.05it/s]
100%|██████████| 88/88 [00:00<00:00, 253.52it/s]                

100%|██████████| 22/22 [00:00<00:00, 511.45it/s]

  0%|          | 0/88 [00:00<?, ?it/s]
  0%|          | 0.2545454545454546/88 [00:00<00:35,  2.50it/s]

...


Train Epoch: 86	Accuracy: 543/702.4000000000001 (77%)


Test Epoch: 86	Accuracy: 128/175.60000000000002 (73%)


Train Epoch: 87 [0/702.4000000000001 (0%)]	Loss: 2.900242



 29%|██▉       | 25.454545454545425/88 [00:00<00:00, 148.19it/s]
100%|██████████| 88/88 [00:00<00:00, 251.34it/s]                

100%|██████████| 22/22 [00:00<00:00, 545.38it/s]

  0%|          | 0/88 [00:00<?, ?it/s]
  0%|          | 0.2545454545454546/88 [00:00<00:35,  2.46it/s]


Train Epoch: 87	Accuracy: 560/702.4000000000001 (80%)


Test Epoch: 87	Accuracy: 122/175.60000000000002 (69%)


Train Epoch: 88 [0/702.4000000000001 (0%)]	Loss: 1.219320



 32%|███▏      | 28.499999999999968/88 [00:00<00:00, 163.30it/s]
100%|██████████| 88/88 [00:00<00:00, 266.05it/s]                

  0%|          | 0/100 [00:33<?, ?it/s]


Train Epoch: 88	Accuracy: 560/702.4000000000001 (80%)


Test Epoch: 88	Accuracy: 122/175.60000000000002 (69%)


 Max Test Accuracy is 73.46241457858768 when epoch is 38
